In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Load data

In [2]:
c_df = pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Ensemble-Machine-Learning-Cookbook/master/Chapter02/Cryotherapy.csv')
w_df = pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Ensemble-Machine-Learning-Cookbook/master/Chapter02/whitewines.csv')
wb_df = pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Ensemble-Machine-Learning-Cookbook/master/Chapter02/wisc_bc_data.csv')

In [4]:
c_df.head()

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
0,1,35,12.00,5,1,100,0
1,1,29,7.00,5,1,96,1
2,1,50,8.00,1,3,132,0
3,1,32,11.75,7,3,750,0
4,1,67,9.25,1,1,42,0


In [31]:
c_df.tail()

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
85,2,34,12.00,3,3,95,0
86,2,20,3.50,6,1,75,1
87,2,35,8.25,8,3,100,0
88,1,24,10.75,10,1,20,1
89,1,19,8.00,8,1,160,1


In [5]:
w_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6.7,0.62,0.24,1.10,0.039,6.0,62.0,0.99340,3.41,0.32,10.400000,5
1,5.7,0.22,0.20,16.00,0.044,41.0,113.0,0.99862,3.22,0.46,8.900000,6
2,5.9,0.19,0.26,7.40,0.034,33.0,123.0,0.99500,3.49,0.42,10.100000,6
3,5.3,0.47,0.10,1.30,0.036,11.0,74.0,0.99082,3.48,0.54,11.200000,4
4,6.4,0.29,0.21,9.65,0.041,36.0,119.0,0.99334,2.99,0.34,10.933333,6


In [32]:
w_df.tail()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4893,6.1,0.28,0.24,19.95,0.074,32.0,174.0,0.99922,3.19,0.44,9.3,6
4894,6.8,0.39,0.34,7.40,0.020,38.0,133.0,0.99212,3.18,0.44,12.0,7
4895,6.9,0.25,0.29,2.40,0.038,28.0,76.0,0.99088,3.01,0.36,11.7,7
4896,7.1,0.26,0.31,2.20,0.044,29.0,128.0,0.99370,3.34,0.64,10.9,8
4897,8.9,0.33,0.32,1.50,0.047,11.0,200.0,0.99540,3.19,0.46,9.4,5


In [6]:
wb_df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,87139402,B,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,0.05955,0.2360,0.6656,1.670,17.43,0.008045,0.011800,0.01683,0.012410,0.01924,0.002248,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,8910251,B,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,0.06491,0.4505,1.1970,3.430,27.10,0.007470,0.035810,0.03354,0.013650,0.03504,0.003318,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,905520,B,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,0.06340,0.1967,1.3870,1.342,13.54,0.005158,0.009355,0.01056,0.007483,0.01718,0.002198,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,868871,B,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,0.06072,0.3384,1.3430,1.851,26.33,0.011270,0.034980,0.02187,0.019650,0.01580,0.003442,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,9012568,B,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,0.05544,0.1783,0.4125,1.338,17.72,0.005012,0.014850,0.01551,0.009155,0.01647,0.001767,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766


In [33]:
wb_df.tail()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
564,911320502,0,13.17,18.22,84.28,537.3,0.07466,0.05994,0.04859,0.02870,0.1454,0.05549,0.2023,0.6850,1.236,16.89,0.005969,0.01493,0.01564,0.008463,0.01093,0.001672,14.90,23.89,95.10,687.6,0.1282,0.1965,0.18760,0.10450,0.2235,0.06925
565,898677,0,10.26,14.71,66.20,321.6,0.09882,0.09159,0.03581,0.02037,0.1633,0.07005,0.3380,2.5090,2.394,19.33,0.017360,0.04671,0.02611,0.012960,0.03675,0.006758,10.88,19.48,70.89,357.1,0.1360,0.1636,0.07162,0.04074,0.2434,0.08488
566,873885,1,15.28,22.41,98.92,710.6,0.09057,0.10520,0.05375,0.03263,0.1727,0.06317,0.2054,0.4956,1.344,19.53,0.003290,0.01395,0.01774,0.006009,0.01172,0.002575,17.80,28.03,113.80,973.1,0.1301,0.3299,0.36300,0.12260,0.3175,0.09772
567,911201,0,14.53,13.98,93.86,644.2,0.10990,0.09242,0.06895,0.06495,0.1650,0.06121,0.3060,0.7213,2.143,25.70,0.006133,0.01251,0.01615,0.011360,0.02207,0.003563,15.80,16.93,103.10,749.9,0.1347,0.1478,0.13730,0.10690,0.2606,0.07810
568,9012795,1,21.37,15.10,141.30,1386.0,0.10010,0.15150,0.19320,0.12550,0.1973,0.06183,0.3414,1.3090,2.407,39.06,0.004426,0.02675,0.03437,0.013430,0.01675,0.004367,22.69,21.84,152.10,1535.0,0.1192,0.2840,0.40240,0.19660,0.2730,0.08666


### Data Preparation

Label encoding

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
wb_df['diagnosis'] = LabelEncoder().fit_transform(wb_df['diagnosis'])

Train / Test Split

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(c_df.drop(['Result_of_Treatment'], axis=1), c_df['Result_of_Treatment'], test_size=0.3, random_state=1)
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(w_df.drop(['quality'], axis=1), w_df['quality'], test_size=0.3, random_state=1)
X_train_wb, X_test_wb, y_train_wb, y_test_wb = train_test_split(wb_df.drop(['diagnosis'], axis=1), wb_df['diagnosis'], test_size=0.3, random_state=1)

### Load Model

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, VotingRegressor

In [18]:
from sklearn.metrics import accuracy_score, mean_squared_error

### Voting

#### Hard voting & Soft voting

In [68]:
def voting_model(models, voting, X_train, X_test, y_train, y_test, case_type='c'):
    for model in models:
        model[1].fit(X_train, y_train)
        y_pred = model[1].predict(X_test)
        if case_type == 'c':
            print(model[1].__class__.__name__, accuracy_score(y_test, y_pred))
        else:
            print(model[1].__class__.__name__, mean_squared_error(y_test, y_pred))
    
    if case_type == 'c':
        ens = VotingClassifier(estimators=models, voting=voting)
        text = f'Classifier ensemble accuracy using {voting} voting:'
        ens.fit(X_train, y_train)
        y_pred = ens.predict(X_test)
        print(f'{text} {accuracy_score(y_test, y_pred)}')
    else:
        ens = VotingRegressor(estimators=models)
        text = f'Regressior ensemble mse using voting:'
        ens.fit(X_train, y_train)
        y_pred = ens.predict(X_test)
        print(f'{text} {mean_squared_error(y_test, y_pred)}')
    
    return ens

### Average

default average and weighted average

In [71]:
def average_model(models, X_train, X_test, y_train, y_test, case_type='c', avg_type='average', weights=None):
    y_all_pred = 0
    if avg_type =='average':
        weights = [w/len(models) for w in range(len(models))]

    for i in range(len(models)):
        models[i][1].fit(X_train, y_train)
        y_pred = models[i][1].predict(X_test)

        y_all_pred += y_pred * weights[i]

    avg_result = y_all_pred / len(models)

    if case_type == 'c':
        text = f'Classifier average accuracy:'
        print(f'{text} {accuracy_score(y_test, avg_result.round(), normalize=True)}')
    else:
        text = f'Regressior average mse:'
        print(f'{text} {mean_squared_error(y_test, avg_result)}')

    return avg_result

### Model Train and Evaluation

#### Cryotherapy data

In [40]:
# Classification Problem
models = []
models.append(('DT', DecisionTreeClassifier(random_state=1)))
models.append(('SVC', SVC(random_state=1, probability=True)))
models.append(('LogitR', LogisticRegression(random_state=1)))

Voting

In [41]:
v_hard_model = voting_model(models, 'hard', X_train_c, X_test_c, y_train_c, y_test_c, case_type='c')

DecisionTreeClassifier 0.8148148148148148
SVC 0.7407407407407407
LogisticRegression 0.8518518518518519
Classifier ensemble accuracy using hard voting: 0.8518518518518519


In [42]:
v_soft_model = voting_model(models, 'soft', X_train_c, X_test_c, y_train_c, y_test_c, case_type='c')

DecisionTreeClassifier 0.8148148148148148
SVC 0.7407407407407407
LogisticRegression 0.8518518518518519
Classifier ensemble accuracy using soft voting: 0.7777777777777778


Average

In [58]:
avg_result = average_model(models, X_train_c, X_test_c, y_train_c, y_test_c, case_type='c', avg_type='average')

Classifier average accuracy: 0.4444444444444444


Weighted Average

In [60]:
weights = [0.4, 0.4, 0.2]
avg_result = average_model(models, X_train_c, X_test_c, y_train_c, y_test_c, case_type='c', avg_type='weighted_average', weights=weights)

Classifier average accuracy: 0.4444444444444444


#### Whitewines Data

In [64]:
# Regression problem
models = []
models.append(('LR', LinearRegression()))
models.append(('SVR', SVR()))
models.append(('DT', DecisionTreeRegressor()))

In [69]:
v__model = voting_model(models, 'hard', X_train_w, X_test_w, y_train_w, y_test_w, case_type='r')

LinearRegression 0.6100009541913035
SVR 0.7262533715955524
DecisionTreeRegressor 0.8108843537414966
Regressior ensemble mse using voting: 0.5328320109844259


In [72]:
avg_result = average_model(models, X_train_w, X_test_w, y_train_w, y_test_w, case_type='r', avg_type='average')

Regressior average mse: 15.822824531981254


In [73]:
weights = [0.4, 0.4, 0.2]
avg_result = average_model(models, X_train_w, X_test_w, y_train_w, y_test_w, case_type='r', avg_type='weighted_average', weights=weights)

Regressior average mse: 15.875417777977155


### WISC Data

In [74]:
# Classification problem
models = []
models.append(('DT', DecisionTreeClassifier(random_state=1)))
models.append(('SVC', SVC(random_state=1, probability=True)))
models.append(('LogitR', LogisticRegression(random_state=1)))

In [75]:
v_hard_model = voting_model(models, 'hard', X_train_wb, X_test_wb, y_train_wb, y_test_wb, case_type='c')
v_soft_model = voting_model(models, 'soft', X_train_wb, X_test_wb, y_train_wb, y_test_wb, case_type='c')

DecisionTreeClassifier 0.935672514619883
SVC 0.6257309941520468
LogisticRegression 0.6783625730994152
Classifier ensemble accuracy using hard voting: 0.695906432748538
DecisionTreeClassifier 0.935672514619883
SVC 0.6257309941520468
LogisticRegression 0.6783625730994152
Classifier ensemble accuracy using soft voting: 0.935672514619883


In [76]:
avg_result = average_model(models, X_train_wb, X_test_wb, y_train_wb, y_test_wb, case_type='c', avg_type='average')

Classifier average accuracy: 0.6783625730994152


In [77]:
weights = [0.4, 0.4, 0.2]
avg_result = average_model(models, X_train_wb, X_test_wb, y_train_wb, y_test_wb, case_type='c', avg_type='weighted_average', weights=weights)

Classifier average accuracy: 0.6783625730994152
